# Import needed modules

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import matplotlib.pyplot as plt
from scipy import stats
import scikit_posthocs as posthocs
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import itertools
import seaborn as sns

# Read HCV_Data_Germany_ngseminar dataset

In [ ]:
hcvDataG = pd.read_csv('HCV_Data_Germany_ngseminar.csv', header = 0, index_col = 0)
display(hcvDataG)

In [ ]:
for el in np.unique(hcvDataG.loc[:, 'Category']):
    print(el)

In [ ]:
for colname in hcvDataG.columns:
    print(colname)

# Read another dataset: HCV_Data_Egypt_ngseminar

In [ ]:
hcvDataE = pd.read_csv('HCV_Data_Egypt_ngseminar.csv', header = 0, index_col = 0)
hcvDataE

In [ ]:
for colname in hcvDataE.columns:
    print(colname)

# Descriptive statistics

**We'll start with Germany dataset**

**Check how many cases and variables are in our dataset**

In [ ]:
nrows, ncolumns = hcvDataG.shape
print('Number of cases (rows): {0:d}'.format(nrows))
print('Number of variables (columns): {0:d}'.format(ncolumns))

**Calculate descriptive statistics for age**

In [ ]:
hcvDataG.loc[:, 'Age'].describe()

**Reapeat calculation dividing patients by gender**

In [ ]:
hcvDataG.loc[:, 'Age'].groupby(hcvDataG.loc[:, 'Sex']).describe()

**Now describe 2 variable at a time: Age and CHOL, divide patients by gender**

In [ ]:
hcvDataG.loc[:, ['Age', 'CHOL']].groupby(hcvDataG.loc[:, 'Sex']).describe()

## Distributions

**Examine visually distribution of CHOL**

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
pd.DataFrame.hist(data = hcvDataG, column = 'CHOL', ax = ax)
ax.grid(False)

Now divide patients by sex

In [ ]:
pd.DataFrame.hist(data = hcvDataG, column = 'CHOL', by='Sex', figsize=(14,5))

**Let's repeat the same with BIL**

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
pd.DataFrame.hist(data = hcvDataG, column = 'BIL', ax = ax)
ax.grid(False)

In [ ]:
pd.DataFrame.hist(data = hcvDataG, column = 'BIL', by='Sex', figsize=(14,5))

**Try to do the same wit ALB dividing patients by Category**

In [ ]:
_ = pd.DataFrame.hist(data = hcvDataG, column = 'ALB', by='Category', layout=(2,2), figsize=(10,10))

# Statistical tests for continous variables

## Normality of distribution

**Check if age of patients is normally distributed in our dataset**

In [ ]:
s, p = stats.shapiro(hcvDataG.loc[:, 'Age'])
print('test statistics: {0:.4f}'.format(s))
print('p-value: {0:.4f}'.format(p))

**Repeat the test grouping patients by gender**

In [ ]:
hcvDataG_by_gender = hcvDataG.groupby('Sex')

for group in hcvDataG_by_gender:
    groupName = group[0]
    groupData = hcvDataG_by_gender.get_group(groupName)
    
    s, p = stats.shapiro(groupData.loc[:, 'Age'])
    
    print('Group: {0:s}'.format(groupName))
    print('  test statistics: {0:.4f}'.format(s))
    print('  tp-value: {0:.4f}'.format(p))
    print()

**Now let's check ALB, ALP, CHOL and CREA, dividing patients by sex**

In [ ]:
hcvDataG_by_stage = hcvDataG.groupby('Sex')
variablesForAnalysis = ['ALB', 'ALP', 'CHOL', 'CREA']

for variable in variablesForAnalysis:
    print(variable)
    for group in hcvDataG_by_stage:
        groupName = group[0]
        groupData = hcvDataG_by_stage.get_group(groupName)

        s, p = stats.shapiro(groupData.loc[:, variable].dropna())

        print('  Group: {0:s}'.format(str(groupName)))
        print('    test statistics: {0:.4f}'.format(s))
        print('    p-value: {0:.4f}'.format(p))
        
    print()

## Difference between 2 independent groups

**Compare ALP between healthy and ill people. Who has higher ALP?**

First, check the assumption for t-test:
* normality of distribution in both groups
* homogeneity of variance (basically equal varience in both groups)

In [ ]:
hcvDataG_by_health = hcvDataG.groupby('Healthy')

for group in hcvDataG_by_health:
    groupName = group[0]
    groupData = hcvDataG_by_health.get_group(groupName)
    
    s, p = stats.shapiro(groupData.loc[:, 'ALP'].dropna())
    
    print('Group: {0:s}'.format(groupName))
    print('  test statistics: {0:.4f}'.format(s))
    print('  p-value: {0:.4f}'.format(p))
    print()

Let's try logarithm of ALP

In [ ]:
hcvDataG_by_health = hcvDataG.groupby('Healthy')

for group in hcvDataG_by_health:
    groupName = group[0]
    groupData = hcvDataG_by_health.get_group(groupName)
    
    s, p = stats.shapiro(np.log(groupData.loc[:, 'ALP'].dropna()))
    
    print('Group: {0:s}'.format(groupName))
    print('  test statistics: {0:.4f}'.format(s))
    print('  p-value: {0:.4f}'.format(p))
    print()

Now, it's fine, we cen move to the next assumption - equal variances.

But before, let's add log ALP as a new variable to our data frame, we will need it.

In [ ]:
hcvDataG.loc[:, 'log ALP'] = np.log(hcvDataG.loc[:, 'ALP'] )

In [ ]:
hcvDataG_by_health = hcvDataG.groupby('Healthy')
  
hcvDataG_healthy_ALP = hcvDataG_by_health.get_group('YES').loc[:, 'log ALP'].dropna()
hcvDataG_ill_ALP = hcvDataG_by_health.get_group('NO').loc[:, 'log ALP'].dropna()

s, p = stats.bartlett(hcvDataG_healthy_ALP, hcvDataG_ill_ALP)

print('test statistics: {0:.4f}'.format(s))
print('p-value: {0:.4f}'.format(p))
print()

Second assumption is not met, so let's check the difference with Mann-Whithey test.

In [ ]:
s, p = stats.mannwhitneyu(hcvDataG_healthy_ALP, hcvDataG_ill_ALP, use_continuity = True, alternative='two-sided')

print('Mann-Whitney test for ALP in healthy and ill people')
print('  test statistics: {0:.4f}'.format(s))
print('  p-value: {0:.4f}'.format(p))

Would the conclusion change if we didn't use log-transform?

In [ ]:
hcvDataG_healthy_ALP_noLog = hcvDataG_by_health.get_group('YES').loc[:, 'ALP'].dropna()
hcvDataG_ill_ALP_noLog = hcvDataG_by_health.get_group('NO').loc[:, 'ALP'].dropna()

s, p = stats.mannwhitneyu(hcvDataG_healthy_ALP_noLog, hcvDataG_ill_ALP_noLog, 
        use_continuity = True, alternative='two-sided')

print('Mann-Whitney test for ALP in healthy and ill people')
print('  test statistics: {0:.4f}'.format(s))
print('  p-value: {0:.4f}'.format(p))

**It's exactly the same. Why?**

**And finally, let's see who has higher ALP.**

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
hcvDataG.boxplot(column = 'ALP', by='Healthy', ax=ax, grid=False)

**Would the conclusion change if we used t-test instead?**

In [ ]:
s, p = stats.ttest_ind(hcvDataG_healthy_ALP, hcvDataG_ill_ALP, nan_policy='omit')

print('Mann-Whitney test for ALP in healthy and ill people')
print('  test statistics: {0:.4f}'.format(s))
print('  p-value: {0:.4f}'.format(p))

In [ ]:
s, p = stats.ttest_ind(hcvDataG_healthy_ALP_noLog, hcvDataG_ill_ALP_noLog, nan_policy='omit')

print('Mann-Whitney test for ALP in healthy and ill people')
print('  test statistics: {0:.4f}'.format(s))
print('  p-value: {0:.4f}'.format(p))

## Differences between 3 or more independent groups

**Check if patients with different 'Category' have differnt ALB**

Start by verification of assumptions for ANOVA

In [ ]:
hcvDataG_by_cat = hcvDataG.groupby('Category')
varName = 'ALP'

for group in hcvDataG_by_cat:
    groupName = group[0]
    groupData = hcvDataG_by_cat.get_group(groupName)
    
    s, p = stats.shapiro(groupData.loc[:, varName].dropna())
    
    print('Group: {0:s}'.format(groupName))
    print('  test statistics: {0:.4f}'.format(s))
    print('  p-value: {0:.4f}'.format(p))
    print()

Normality assumption is not met, so let's use Kruskal-Wallis test to compare the groups.

In [ ]:
hcvDonors = hcvDataG_by_cat.get_group('Blood Donor').loc[:, varName]
hcvCirrhosis = hcvDataG_by_cat.get_group('Cirrhosis').loc[:, varName]
hcvFibrosis = hcvDataG_by_cat.get_group('Fibrosis').loc[:, varName]
hcvHepatitis = hcvDataG_by_cat.get_group('Hepatitis').loc[:, varName]

s, p = stats.kruskal(hcvDonors, hcvCirrhosis, hcvFibrosis, hcvHepatitis, nan_policy = 'omit')

print('test statistics: {0:.4f}'.format(s))
print('p-value: {0:.4f}'.format(p))

Which pairs of groups differ?

In [ ]:
post_hoc_results = posthocs.posthoc_dunn(hcvDataG, val_col = varName, group_col = 'Category')
post_hoc_results

In [ ]:
group_names = list(np.unique(hcvDataG.loc[:, 'Category']))

for comb in itertools.combinations(group_names, r=2):
    names = list(comb)
    print('{0:s} vs {1:s}: p={2:.4f}'.format(names[0], names[1], post_hoc_results.loc[names[0], names[1]]))

Finally, let's look at the differences

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
hcvDataG.boxplot(column = varName, by='Category', ax=ax, grid=False)
# ax.set_ylim(15,65)

**What would happen with ANOVA applied to the task?**

In [ ]:
s, p = stats.f_oneway(
    hcvDonors.dropna(),
    hcvCirrhosis.dropna(),
    hcvFibrosis.dropna(),
    hcvHepatitis.dropna(),
)

print('test statistics: {0:.4f}'.format(s))
print('p-value: {0:.4f}'.format(p))

In [ ]:
post_hoc_results = posthocs.posthoc_tukey(hcvDataG, val_col=varName, group_col='Category')
post_hoc_results

In [ ]:
group_names = list(np.unique(hcvDataG.loc[:, 'Category']))

for comb in itertools.combinations(group_names, r=2):
    names = list(comb)
    print('{0:s} vs {1:s}: p={2:.4f}'.format(names[0], names[1], post_hoc_results.loc[names[0], names[1]]))

## Differences between 2 dependent groups

**Now, we will move to the segond (Egyptian) dataset.**

**Check is amount of viral RNA changed between the baseline (RNA Base) and end of treatment (RNA EOT)**

Start checking assumptions of parametric (t-test for related/dependant).

In [ ]:
varNames = ['RNA Base', 'RNA EOT']
hcvRNA = hcvDataE.loc[:, varNames].dropna(axis='index', how='any')

In [ ]:
for varName in varNames:
    s, p = stats.shapiro(hcvRNA.loc[:, varName])

    print('Group: {0:s}'.format(varName))
    print('  test statistics: {0:.4f}'.format(s))
    print('  p-value: {0:.4f}'.format(p))
    print()

At this point, you can check if logarithmic transformation of values makes distribution normal (buyt here it doesn't).

To analyze the changes in viral RNA, we will use non-parametric test: Wilcoxon test.

In [ ]:
s, p = stats.wilcoxon(hcvRNA.loc[:, 'RNA Base'], hcvRNA.loc[:, 'RNA EOT'])

print('test statistics: {0:.4f}'.format(s))
print('p-value: {0:.4f}'.format(p))

Amount of viral DNA changes during treatment, so try to visualize it.

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
hcvRNA.boxplot(column = varNames, ax=ax, grid=False)

**What if we used t-test instead?**

In [ ]:
s, p = stats.ttest_rel(hcvRNA.loc[:, 'RNA Base'], hcvRNA.loc[:, 'RNA EOT'])

print('test statistics: {0:.4f}'.format(s))
print('p-value: {0:.4f}'.format(p))

## Differences between 3 or more related measurments

**Analyse changes in amounts viral RNA along the treatment**

We check normality of distribution at baseline at and at the end of treatment, so we know that assumptions for ANOVA are not met. We can move directly to Friedman test.

In [ ]:
varNames = ['RNA Base', 'RNA 4', 'RNA 12', 'RNA EOT']
hcvRNA = hcvDataE.loc[:, varNames].dropna(axis='index', how='any')

In [ ]:
s, p = stats.friedmanchisquare(
    hcvRNA.loc[:, 'RNA Base'],
    hcvRNA.loc[:, 'RNA 4'],
    hcvRNA.loc[:, 'RNA 12'],
    hcvRNA.loc[:, 'RNA EOT'],
)

print('test statistics: {0:.4f}'.format(s))
print('p-value: {0:.4f}'.format(p))

In [ ]:
post_hoc_results = posthocs.posthoc_nemenyi_friedman(hcvRNA, y_col=varNames)
post_hoc_results.loc[varNames, varNames]

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
_ = hcvRNA.boxplot(column = varNames, ax=ax, grid=False)
_ = ax.set_ylim(top=15*10**5)

In [ ]:
hcvRNA.loc[:, 'Patient'] = hcvRNA.index
hcvRNAmelted = pd.melt(hcvRNA, id_vars = 'Patient', value_vars = varNames, var_name = 'Timepoint', value_name = 'RNA')
hcvRNAmelted

In [ ]:
patientIDs = np.unique(hcvRNAmelted['Patient'])
selectedPatientIds = np.random.choice(patientIDs, size=10, replace=False)

selectedRows = [id in list(selectedPatientIds) for id in hcvRNAmelted['Patient']]

hcvRNAmelted_some_patients = hcvRNAmelted.loc[selectedRows,:]
hcvRNAmelted_some_patients

ax = sns.pointplot(data=hcvRNAmelted_some_patients, x='Timepoint', y='RNA', hue='Patient', palette='colorblind',
                  legend=False)
ax.get_legend().remove()

**Would the conclusion change if we applied ANOVA for repeated measurments?**

In [ ]:
anova_rm_rna = AnovaRM(hcvRNAmelted, depvar = 'RNA', subject = 'Patient', within = ['Timepoint'])
anova_rm_rna_res = anova_rm_rna.fit()

print(anova_rm_rna_res)

In [ ]:
post_hoc_res = pairwise_tukeyhsd(hcvRNAmelted.loc[:, 'RNA'], groups = hcvRNAmelted.loc[:, 'Timepoint'])
post_hoc_res.summary()

# Correlations

**Check if laboratory results at different timepoints are correlated**

TODO: choose only some of them!!!

In [ ]:
varNames = ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']
hcvLab = hcvDataG.loc[:, varNames]
# hcvRNA = hcvDataE.loc[:, varNames].dropna(axis='index', how='any')

In [ ]:
hcvLab.corr(method='spearman')

In [ ]:
hcvLab.corr(method='pearson')

In [ ]:
hcvLab

**If you want also p-values, you need scipy**

In [ ]:
r, p = stats.spearmanr(hcvLab, nan_policy='omit')

display(pd.DataFrame(r, index=varNames, columns=varNames))

display(pd.DataFrame(p, index=varNames, columns=varNames))

But the same doesn't work with Pearson correlation...

In [ ]:
r = pd.DataFrame(np.zeros([len(varNames),len(varNames)]), columns=varNames, index=varNames)
p = pd.DataFrame(np.zeros([len(varNames),len(varNames)]), columns=varNames, index=varNames)

for comb in itertools.combinations(varNames, r=2):
    names = list(comb)
    name1 = names[0]
    name2 = names[1]
    hcvLabCurrent = hcvLab.loc[:, [name1, name2]].dropna()
    cr, cp = stats.pearsonr(hcvLabCurrent.loc[:, name1], hcvLabCurrent.loc[:, name2])
    r.loc[name1, name2] = r.loc[name2, name1] = cr
    p.loc[name1, name2] = p.loc[name2, name1] = cp

display(r)

display(p)

Let's visualize the correlation between CHOL and ALT

In [ ]:
hcvLab.plot.scatter(x='CHOL', y='PROT')

Let's look at all correlations at once

In [ ]:
_ = sns.pairplot(hcvLab)

In [ ]:
help(sns.pairplot)

Repeat this type of analysis for viral RNA variables in Egyptian dataset. Don't forget the plots.

Can you tell what is wrong in that analysis?

# Statistical tests for nominal variables

**Using Egyptian dataset, check if Nausea/Vomiting is equaly common in patients with and without Epigastric pain**

In [ ]:
hcvDataE.columns

In [ ]:
table1 = pd.crosstab(hcvDataE['Epigastric pain'], hcvDataE['Nausea/Vomiting'])
table1

In [ ]:
table1_percent_rows = table1.apply(lambda r: r/r.sum()*100, axis=1)
table1_percent_rows

In [ ]:
table1_percent_cols = table1.apply(lambda r: r/r.sum()*100, axis=0)
table1_percent_cols

In [ ]:
s, p, _, _ = stats.chi2_contingency(table1)
OR, pf = stats.fisher_exact(table1)
print(p)
print(pf)
print()

**Now, check if presence of Epigastric pain is related to Baseline histological staging**

In [ ]:
table2 = pd.crosstab(hcvDataE['Baseline histological staging'], hcvDataE['Epigastric pain'])
display(table2)
s, p, _, _ = stats.chi2_contingency(table2)
print(p)
print()

In [ ]:
table2_percent_rows = table2.apply(lambda r: r/r.sum()*100, axis=1)
table2_percent_rows

In [ ]:
table2_percent_cols = table2.apply(lambda r: r/r.sum()*100, axis=0)
table2_percent_cols

# Multiple testing